# 克隆仓库

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive
!git clone https://github.com/Jimmy142857/Asymmetric_DECA.git
%cd /content/drive/MyDrive/Asymmetric_DECA

In [ ]:
# 装FAN
!pip install face-alignment

In [ ]:
%cd /content/drive/MyDrive/Asymmetric_DECA

# 嘴角拉伸

In [ ]:
# 针对DECA输出进行剪裁并保存
import cv2

img_path = '/content/drive/MyDrive/Asymmetric_DECA/Palsy_37/palsy_37_vis_original_size.jpg'        # DECA输出图片地址
ori_path = '/content/drive/MyDrive/Asymmetric_DECA/Palsy_37/ori.png'                    # 剪裁后人脸图片
fit_path = '/content/drive/MyDrive/Asymmetric_DECA/Palsy_37/fit.png'                    # 剪裁后嵌入图片

img = cv2.imread(img_path)
width = int(img.shape[1] / 5)                 # 单张图片的宽度

ori_pic = img[:, :width]                    # 原始图片
fit_pic = img[:, width * 3 : width * 4]          # 嵌入图片

cv2.imwrite(ori_path, ori_pic)                  # 保存图片
cv2.imwrite(fit_path, fit_pic)

In [ ]:
# 针对剪裁结果进行关键点检测
import face_alignment
import numpy as np
import cv2

# 保存路径
lmk_ori = '/content/drive/MyDrive/Asymmetric_DECA/Palsy_37/ori.npy'
lmk_fit = '/content/drive/MyDrive/Asymmetric_DECA/Palsy_37/fit.npy'

# FAN初始化
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False, device='cpu')

# 剪裁图片读取
img_ori = cv2.imread(ori_path)
img_fit = cv2.imread(fit_path)

# 得到人脸关键点
preds_ori = fa.get_landmarks_from_image(img_ori)
preds_ori = np.array(preds_ori)
preds_ori = np.reshape(preds_ori,(68,2))      # 三维数组转化为二维

# 得到模型关键点
preds_fit = fa.get_landmarks_from_image(img_fit)
preds_fit = np.array(preds_fit)
preds_fit = np.reshape(preds_fit,(68,2))

# 保存
np.save(lmk_ori, preds_ori)
np.save(lmk_fit, preds_fit)

In [ ]:
# OBJ文件读取
import numpy as np
from obj_op import OBJ

filename = 'palsy_37.obj'                                                                 # 文件名
fdir = '/content/drive/MyDrive/Asymmetric_DECA/Palsy_37/'           # 文件夹 

FLAME = OBJ(fdir, filename)

In [ ]:
# 创建模型所有顶点、面的数组 (OBJ中顶点和面的序号从零开始，面中存储的顶点序号从一开始)
faces = []
for x in FLAME.faces:
    faces.append(x[0])

faces = np.array(faces)
vertices = np.array(FLAME.vertices)

In [ ]:
# 得到嘴角点、嘴唇点、嘴角点位移信息
from obj_op import Mov_corner

Mov_info = Mov_corner(lmk_ori, lmk_fit, vertices)

corner = np.atleast_1d( Mov_info['Corner_point'] )        # 嘴角点序号
lip = np.atleast_1d( Mov_info['Lip_point'] )           # 嘴唇点序号
mov_corner = Mov_info['Mov']                    # 嘴角点位移

lip_ori = vertices[lip]                      # 原始嘴唇点位置

In [ ]:
# 寻找嘴角外层顶点
from obj_op import find_vertices

layer = 12                               # 外侧点层数 (嘴角12最优、下嘴唇偏侧7最优)
round_corner = []                                    
round_corner.append(list(corner))                     # 存储最初的嘴角点
for i in range(layer):                         # 记录外侧点
    round_corner.append(find_vertices(faces, corner))
    corner = list(corner) + round_corner[i + 1]

In [ ]:
# 根据衰减算法N次迭代更新顶点位置
from obj_op import damping_mov

N = 10
position = 'corner'

damping_mov(N, mov_corner, round_corner, vertices, position)

In [ ]:
# 保存文件
import os
from obj_op import save_obj

os.makedirs('/content/drive/MyDrive/Asymmetric_DECA/Palsy_37/results', exist_ok = True)

file_path = '/content/drive/MyDrive/Asymmetric_DECA/Palsy_37/results/test1.obj'

save_obj(file_path, vertices, faces)

# 嘴唇回拉

In [ ]:
# 在第一次拉伸嘴角的基础上进行
lip_ori = lip_ori
lip_cur = vertices[lip]

mov_lip = lip_ori - lip_cur                   # 嘴唇点初始位移
mov_lip = np.reshape(mov_lip, (3,))                # 维度转化

In [ ]:
# 寻找嘴唇外侧有连接的顶点
from obj_op import find_vertices

layer = 7                                  # 外侧点层数 (嘴角12最优、下嘴唇偏侧7最优)
round_lip = []                                    
round_lip.append(list(lip))                          # 存储最初的嘴唇点
for i in range(layer):                           # 记录外侧点
    round_lip.append(find_vertices(faces, lip))
    lip = list(lip) + round_lip[i + 1]

In [ ]:
# 根据衰减算法N次迭代更新顶点位置
from obj_op import damping_mov

N = 10
position = 'lip'

damping_mov(N, mov_lip, round_lip, vertices, position)

In [ ]:
# 保存文件

from obj_op import save_obj

file_path = '/content/drive/MyDrive/Asymmetric_DECA/Palsy_37/results/test2.obj'

save_obj(file_path, vertices, faces)

# 结果可视化

In [ ]:
!pip install trimesh
!pip install obj2html

In [ ]:
# obj文件转html
from obj2html import obj2html
from IPython.display import display, HTML

file = '/content/drive/MyDrive/Asymmetric_DECA/Palsy_37/results/test1.obj'
obj2html(file, 'index.html')

display(HTML('index.html'))

In [ ]:
# trimesh仓库
import trimesh
import numpy as np
 
file = '/content/drive/MyDrive/Asymmetric_DECA/Palsy_37/palsy_37.obj'
mesh = trimesh.load(file)
v = mesh.vertices
f = mesh.faces

mesh.show()


In [ ]:
file_1 = '/content/drive/MyDrive/Asymmetric_DECA/Palsy_37/results/test2.obj'
mesh = trimesh.load(file_1)

mesh.show()

In [ ]:
file_2 = '/content/drive/MyDrive/Asymmetric_DECA/Palsy_37/results/test2.obj'
mesh = trimesh.load(file_2)

mesh.show()